Using OAI Function Calling

In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../creds/.env")

True

In [16]:
from openai import AzureOpenAI

azure_endpoint = "https://cursor-gpt-4.openai.azure.com"
api_version="2024-02-15-preview"


client = AzureOpenAI(
        azure_endpoint=azure_endpoint,
        api_version=api_version,
    )

In [17]:
# THREE types of tools: Code Interpreter, Retrieval, Function Calling
tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]

In [18]:
messages = [
    {"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
    {"role": "user", "content": "What's the weather like today in Seattle?"}
]

chat_completion = client.chat.completions.create(
    model="pjf-dpo-turbo-35",
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, or none
)
print(chat_completion)

ChatCompletion(id='chatcmpl-8vFihbxIPrB8ib9zfd69rt870LoiJ', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_L8SgNpYAuYUI3LzPQiG307GF', function=Function(arguments='{"location":"Seattle","unit":"celsius"}', name='get_current_weather'), type='function')]), content_filter_results={})], created=1708655815, model='gpt-35-turbo', object='chat.completion', system_fingerprint='fp_68a7d165bf', usage=CompletionUsage(completion_tokens=20, prompt_tokens=105, total_tokens=125), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}])


In [ ]:
react_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris
""".strip()

In [ ]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})
    
    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        # deployment_name = "cursor-gpt-4"
        deployment_name = "pjf-dpo-turbo-35"

        completion = client.chat.completions.create(
            model=deployment_name,  # e.g. gpt-35-instant
            messages=self.messages,
            # tools=functions,
        )
        # Uncomment this to print out token usage each time, e.g.
        # {"completion_tokens": 86, "prompt_tokens": 26, "total_tokens": 112}
        # print(completion.usage)
        return completion.choices[0].message.content

In [ ]:
# def wikipedia(q):
#     return httpx.get("https://en.wikipedia.org/w/api.php", params={
#         "action": "query",
#         "list": "search",
#         "srsearch": q,
#         "format": "json"
#     }).json()["query"]["search"][0]["snippet"]


# def simon_blog_search(q):
#     results = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
#         "sql": """
#         select
#           blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
#           blog_entry.created
#         from
#           blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
#         where
#           blog_entry_fts match escape_fts(:q)
#         order by
#           blog_entry_fts.rank
#         limit
#           1""".strip(),
#         "_shape": "array",
#         "q": q,
#     }).json()
#     return results[0]["text"]

def calculate(what):
    return eval(what)

known_actions = {
    # "wikipedia": wikipedia,
    "calculate": calculate,
    # "simon_blog_search": simon_blog_search
}

In [ ]:
import re 

action_re = re.compile('^Action: (\w+): (.*)$')
# action_input_regex = re.compile('^Action Input: (\w+): (.*)$')

def query(question, max_turns=5):
    i = 0
    bot = Agent(react_prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        print("Actions Retrieved", actions)
        # action_inputs = [action_input_regex.match(a) for a in result.split('\n') if action_input_regex.match(a)]
        # print("Action Inputs", action_inputs)
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [ ]:
query("what is 2 * twenty five")

TODO:
* ask user for clarification after single ReAct